In [1]:
from langchain_g4f import G4FLLM
from g4f import Provider, models
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import faiss
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [2]:
template = """
Vous es Seth Rensei, un assistant routier. Vpus devriez répondre à la question de la fin en utilisant le contexte suivant.
Si la réponse n'est pas possible à retirer dans le contexte, dites simplement que vous ne connaissez pas.
N'essayez pas d'inventer de réponse.

Context : {context}\n

Question : {question}\n

"""

prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

In [3]:
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name='./LocalModel',
    model_kwargs=model_kwargs,
)

e:\Programs\Py\IhmSqr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loaded_vectors=faiss.FAISS.load_local("./vectors", embeddings, allow_dangerous_deserialization=True)
retriever = loaded_vectors.as_retriever()

In [5]:
parser = StrOutputParser()
result = RunnableParallel(context = retriever,question = RunnablePassthrough())

In [6]:
gpt_llm: LLM = G4FLLM(
        model=models.gpt_35_turbo,
        provider=Provider.Aichatos,
    )

In [7]:
chain2 = result | prompt | gpt_llm | parser

In [12]:
import gradio as gr

from Voice import Voice

In [13]:
vc = Voice()

In [14]:
def qa(question):
    vc.say(chain2.invoke(question))
    return chain2.invoke(question)

In [15]:
inter = gr.Interface(
    fn=qa,
    inputs=gr.Textbox(label='Votre question'),
    outputs='text',
    title='IHM - SQR',
    theme=gr.themes.Glass()
)

In [17]:
if __name__ == '__main__':
    inter.close()
    inter.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
